In [61]:
import sqlite3
import pandas as pd

In [62]:
CHEMBL_DB_PATH = "../raw-data/chembldb/chembl_35/chembl_35_sqlite/chembl_35.db"

In [76]:
SQL_DRUG_INDICATION_QUERY = """
    SELECT comp.molregno, comp.canonical_smiles, mol.indication_class, rec.record_id, ind.mesh_heading, ind.max_phase_for_ind
    FROM compound_structures as comp
    INNER JOIN molecule_dictionary as mol
    ON comp.molregno = mol.molregno
    INNER JOIN compound_records as rec
    on mol.molregno = rec.molregno
    INNER JOIN drug_indication as ind
    ON rec.record_id = ind.record_id
    """

## Reading Directly with SQL

In [77]:
# Establish a connection to the DB and a cursor
con = sqlite3.connect(CHEMBL_DB_PATH)
cur = con.cursor()

In [78]:
res = cur.execute(
    SQL_DRUG_INDICATION_QUERY
)

In [79]:
for i, r in enumerate(res):
    print(r)
    if i > 10:
        break

(438435, 'CC(=O)O[C@H]1CC[C@@]2(C)C(=CC[C@@H]3[C@@H]2CC[C@]2(C)C(c4cccnc4)=CC[C@@H]32)C1', None, 1344515, 'Prostatic Neoplasms', 4)
(448986, 'C[C@H]1O[C@H](O[C@H]2[C@H](O)[C@@H](O)[C@@H](O[C@H]3[C@H](O)[C@@H](O)[C@H](O)O[C@@H]3CO)O[C@@H]2CO)[C@H](O)[C@@H](O)[C@@H]1N[C@H]1C=C(CO)[C@@H](O)[C@H](O)[C@H]1O', 'Inhibitor (alpha-glucosidase)', 1343309, 'Diabetes Mellitus', 4)
(448986, 'C[C@H]1O[C@H](O[C@H]2[C@H](O)[C@@H](O)[C@@H](O[C@H]3[C@H](O)[C@@H](O)[C@H](O)O[C@@H]3CO)O[C@@H]2CO)[C@H](O)[C@@H](O)[C@@H]1N[C@H]1C=C(CO)[C@@H](O)[C@H](O)[C@H]1O', 'Inhibitor (alpha-glucosidase)', 1343309, 'Metabolic Syndrome', 3)
(448986, 'C[C@H]1O[C@H](O[C@H]2[C@H](O)[C@@H](O)[C@@H](O[C@H]3[C@H](O)[C@@H](O)[C@H](O)O[C@@H]3CO)O[C@@H]2CO)[C@H](O)[C@@H](O)[C@@H]1N[C@H]1C=C(CO)[C@@H](O)[C@H](O)[C@H]1O', 'Inhibitor (alpha-glucosidase)', 1343309, 'Non-alcoholic Fatty Liver Disease', 2)
(16450, 'CC(=O)Nc1ccc(O)cc1', 'Analgesic; Antipyretic', 1343835, 'Fever', 4)
(16450, 'CC(=O)Nc1ccc(O)cc1', 'Analgesic; Antipyretic'

In [80]:
query_results = res.fetchall()

In [81]:
len(query_results)

42045

## Read into Pandas

In [86]:
pd_db = pd.read_sql_query(sql=SQL_DRUG_INDICATION_QUERY, con=con)

In [87]:
pd_db

,molregno,canonical_smiles,indication_class,record_id,mesh_heading,max_phase_for_ind
0,438435,CC(=O)O[C@H]1CC[C@@]2(C)C(=CC[C@@H]3[C@@H]2CC[...,None,1344515,Prostatic Neoplasms,4.0
1,448986,C[C@H]1O[C@H](O[C@H]2[C@H](O)[C@@H](O)[C@@H](O...,Inhibitor (alpha-glucosidase),1343309,Diabetes Mellitus,4.0
2,448986,C[C@H]1O[C@H](O[C@H]2[C@H](O)[C@@H](O)[C@@H](O...,Inhibitor (alpha-glucosidase),1343309,Metabolic Syndrome,3.0
3,448986,C[C@H]1O[C@H](O[C@H]2[C@H](O)[C@@H](O)[C@@H](O...,Inhibitor (alpha-glucosidase),1343309,Non-alcoholic Fatty Liver Disease,2.0
4,16450,CC(=O)Nc1ccc(O)cc1,Analgesic; Antipyretic,1343835,Fever,4.0
...,...,...,...,...,...,...
42052,2382487,O=C1Nc2ccccc2C(c2ccccc2)=N[C@@H]1Nc1nnc(-c2ncc...,None,3954671,Virus Diseases,2.0
42053,2832083,CC(C)C[C@@H](C(=O)N1C[C@]2(C[C@H]1C#N)C(=O)Nc1...,None,3955999,Virus Diseases,2.0
42054,1927196,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O...,None,3956425,Inflammation,2.0
42055,2832300,C=CC(=O)N1C[C@H](Nc2ncnc3[nH]ccc23)CC[C@@H]1C....,None,3956275,Alopecia Areata,4.0


In [88]:
pd_db = pd_db[pd_db.max_phase_for_ind >= 3].drop(columns=['molregno', 'record_id', 'max_phase_for_ind', 'indication_class'])

In [89]:
pd.get_dummies(pd_db, columns=['mesh_heading'])

,canonical_smiles,mesh_heading_AIDS Dementia Complex,mesh_heading_AIDS-Associated Nephropathy,mesh_heading_AIDS-Related Complex,mesh_heading_Abdominal Abscess,mesh_heading_Abdominal Neoplasms,mesh_heading_Abdominal Pain,"mesh_heading_Abortion, Habitual","mesh_heading_Abortion, Spontaneous",mesh_heading_Abruptio Placentae,...,mesh_heading_Wrist Fractures,"mesh_heading_Xanthomatosis, Cerebrotendinous",mesh_heading_Xerostomia,mesh_heading_Yaws,mesh_heading_Zollinger-Ellison Syndrome,mesh_heading_Zygomycosis,mesh_heading_alpha-Thalassemia,mesh_heading_beta-Thalassemia,mesh_heading_von Hippel-Lindau Disease,mesh_heading_von Willebrand Diseases
0,CC(=O)O[C@H]1CC[C@@]2(C)C(=CC[C@@H]3[C@@H]2CC[...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,C[C@H]1O[C@H](O[C@H]2[C@H](O)[C@@H](O)[C@@H](O...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,C[C@H]1O[C@H](O[C@H]2[C@H](O)[C@@H](O)[C@@H](O...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,CC(=O)Nc1ccc(O)cc1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,CC(=O)Nc1ccc(O)cc1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41919,COc1ncc(-c2ccc(Cl)cc2)cc1[C@@H](c1ccccc1)[C@@]...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
41943,C=CC(=O)Nc1cc(Nc2nccc(-c3cn(C)c4ccccc34)n2)c(O...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
41957,[O-][n+]1c(CO)c(CO)[n+]([O-])c2ccccc21,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
41959,N#Cc1ccc(N2N=C3c4ccc(C(=O)N5CCC(O)CC5)nc4CC[C@...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
